In [20]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time
import folium
import collections
from collections import OrderedDict
import csv

driver = webdriver.Chrome('../chromedriver.exe') # 이건 우리 빅데이터 강의 자료 6장 20p 보면 다운받는 url있음. 경로 잘 확인해서 테스트하기.
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

# 스타벅스 매장찾기에서 지역 검색 탭 클릭
time.sleep(1)
driver.find_element(By.LINK_TEXT, '지역 검색').click()
time.sleep(4)


# 모든 지역 리스트 만들기
sido_list = driver.find_elements(By.CLASS_NAME, 'set_sido_cd_btn')


# csv로 다운받을 데이터 담는 리스트
starbucks = []

for item in sido_list:
    item.click() # 지역 버튼 클릭
    time.sleep(4)

    # 17은 세종인데 얘네는 낱개로 있어서 전체 버튼이 없음. 그래서 따로 분류함
    if '17' == item.get_attribute('data-sidocd'):

        # 페이지 정보 가져오기
        html = driver.page_source

        # BeautifulSoup 객체로 변환
        soup = BeautifulSoup(html)
        store_info = soup.select('li[data-name]')
        for info in store_info:
            lat = info['data-lat'] # 위도
            long = info['data-long'] # 경도
            name = info['data-name'] # 매장명
            address = info.select_one('p').text.split('1522-3232')[0] 
            starbucks.append({'name': name, 'address': address, 'lat': lat, 'long': long})
        time.sleep(1)

        driver.close()
    else:
        list_all_btn = driver.find_element(By.LINK_TEXT, '전체')
        list_all_btn.click()
        driver.implicitly_wait(5)
        time.sleep(1)

        html = driver.page_source

        soup = BeautifulSoup(html)
        store_info = soup.select('li[data-name]')
        for info in store_info:
            lat = info['data-lat']
            long = info['data-long']
            name = info['data-name']
            address = info.select_one('p').text.split('1522-3232')[0]
            starbucks.append({'name': name, 'address': address, 'lat': lat, 'long': long})
        time.sleep(1)
        
        # 다른 지역 선택하기 위해서 지역 검색 탭 누르기
        driver.find_element(By.LINK_TEXT, '지역 검색').click()
        time.sleep(4)
        
starbucks = [x for i, x in enumerate(starbucks) if x not in starbucks[:i]]

df = pd.DataFrame(starbucks)
df.to_csv('starbucks.csv', index=False, encoding='utf-8') # CSV 파일로 저장

/var/folders/13/lngvcc9153b7xk5c_vj9hm2r0000gn/T/ipykernel_24086/2233694071.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../chromedriver.exe') # 이건 우리 빅데이터 강의 자료 6장 20p 보면 다운받는 url있음


In [21]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time
import folium
import collections
from collections import OrderedDict
import csv


In [22]:
starbucks_file = pd.read_csv('./starbucks.csv', engine='python')

len(starbucks_file) #확인용 출력

1825

In [23]:
map_CB = folium.Map(location=[37.560284, 126.975334], zoom_start = 15)

for i, store in starbucks_file.iterrows():
    folium.Marker(location=[store['lat'], store['long']], popup= store['name'], icon=folium.Icon(color='red', icon='star')).add_to(map_CB)


In [24]:
map_CB

### 스타벅스 매장 주소로 이디야 검색 데이터 만들기

In [25]:
# 이디야는 행정구역 표준이 아님. 따라서 데이터 수정이 필요함.
# 범위가 넓은 지역은 범위를 좁혀달라는 알림창이 뜸.
# 경기도는 시까지 적어야 검색이 가능.

sigungu_list = []

for i, store in starbucks_file.iterrows():
    sido = store['address'].split()[0]
    sigungu = store['address'].split()[1]
    if sido in ("제주특별자치도", "세종특별자치시"): # 얘네는 데이터가 많이 없어서 해당 키워드로만 검색하게 함.
        sigungu_list.append(sido)
    elif sido == "경기도":
        sigungu_list.append(sido + " " + sigungu + " " + store['address'].split()[2])
    else:
        sigungu_list.append(sido + " " + sigungu)

unique_sigungu_list = [x for i, x in enumerate(sigungu_list) if x not in sigungu_list[:i]]
unique_sigungu_new_list = []

for i in unique_sigungu_list:
    addr = i.split()
    if addr[0] == "서울특별시" : addr[0] = "서울"
    elif addr[0] == "부산광역시" : addr[0] = "부산"
    elif addr[0] == "인천광역시" : addr[0] = "인천"
    elif addr[0] == "광주광역시" : addr[0] = "광주"
    elif addr[0] == "대전광역시" : addr[0] = "대전"
    elif addr[0] == "울산광역시" : addr[0] = "울산"
    elif addr[0] == "경기도" : addr[0] = "경기"
    elif addr[0] == "충청북도" : addr[0] = "충북"
    elif addr[0] == "충청남도" : addr[0] = "충남"
    elif addr[0] == "전라북도" : addr[0] = "전북"
    elif addr[0] == "전라남도" : addr[0] = "전남"
    elif addr[0] == "경상북도" : addr[0] = "경북"
    elif addr[0] == "경상남도" : addr[0] = "경남"
    
    if len(addr) < 2:
        unique_sigungu_new_list.append(addr[0])
    elif addr[0] == "경기":
        unique_sigungu_new_list.append(addr[0] + " " + addr[1] + " " + addr[2])
    else:
        unique_sigungu_new_list.append(addr[0] + " " + addr[1])
    
unique_sigungu_new_list



['서울 강남구',
 '서울 강북구',
 '서울 강서구',
 '서울 관악구',
 '서울 광진구',
 '서울 금천구',
 '서울 노원구',
 '서울 도봉구',
 '서울 동작구',
 '서울 마포구',
 '서울 서대문구',
 '서울 서초구',
 '서울 성북구',
 '서울 송파구',
 '서울 양천구',
 '서울 영등포구',
 '서울 은평구',
 '서울 종로구',
 '서울 중구',
 '서울 강동구',
 '서울 구로구',
 '서울 동대문구',
 '서울 성동구',
 '서울 용산구',
 '서울 중랑구',
 '경기 가평군 청평면',
 '경기 가평군 가평읍',
 '경기 고양시 일산동구',
 '경기 과천시 중앙로',
 '경기 과천시 별양상가3로',
 '경기 광명시 범안로',
 '경기 광명시 철산로',
 '경기 광명시 양지로',
 '경기 광명시 오리로',
 '경기 광명시 금하로',
 '경기 광명시 광명로',
 '경기 광명시 디지털로',
 '경기 광명시 일직로',
 '경기 광주시 경충대로',
 '경기 광주시 오포읍',
 '경기 광주시 경안천로',
 '경기 광주시 광주대로',
 '경기 광주시 초월읍',
 '경기 광주시 곤지암읍',
 '경기 구리시 경춘북로',
 '경기 구리시 검배로',
 '경기 구리시 장자대로',
 '경기 구리시 건원대로',
 '경기 구리시 체육관로',
 '경기 구리시 벌말로',
 '경기 군포시 고산로701',
 '경기 군포시 산본로343번길',
 '경기 군포시 번영로',
 '경기 군포시 산본로',
 '경기 군포시 엘에스로',
 '경기 남양주시 다산지금로',
 '경기 남양주시 진접읍',
 '경기 남양주시 다산중앙로123번길',
 '경기 남양주시 미금로',
 '경기 남양주시 와부읍',
 '경기 남양주시 별내5로5번길',
 '경기 남양주시 경춘로',
 '경기 남양주시 별내중앙로',
 '경기 남양주시 늘을1로16번길',
 '경기 남양주시 화도읍',
 '경기 남양주시 다산중앙로',
 '경기 남양주시 도농로',
 '경기 남양주시 덕송2로',
 '경기 남양주시 경강로',
 '경기

In [19]:
driver = webdriver.Chrome('../chromedriver.exe')
url = 'https://www.ediya.com/contents/find_store.html#c'
driver.get(url)

time.sleep(1)
#이디야 주소탭 클릭
driver.find_element(
    By.CSS_SELECTOR, "#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a"
).click()
time.sleep(4)

ediya = []

for item in unique_sigungu_new_list:
    #검색창 지우기
    driver.find_element(By.CSS_SELECTOR, "#keyword").clear()
    # 스타벅스 데이터에서 가져온 주소 입력하기
    driver.find_element(By.CSS_SELECTOR, "#keyword").send_keys(item)
    # 검색 아이콘 클릭
    driver.find_element(By.CSS_SELECTOR, "#keyword_div > form > button").click()
    # 검색결과 없을 때 얼럿창 예외처리
    try: 
        alert = WebDriverWait(driver, 3).until(EC.alert_is_present())
        alert.accept()
    except:
        pass
    driver.implicitly_wait(5)
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html)
    
    # 정보 저장
    placesList = soup.select("#placesList > li")
    for place in placesList:
        name = place.select_one("dt").text
        address = place.select_one("dd").text
        gu = address.split()[1]
        ediya.append({'name': name, 'address': address, 'gu': gu})
    time.sleep(1)

# 중복된 데이터 있으면 제거
ediya = [x for i, x in enumerate(ediya) if x not in ediya[:i]]

df = pd.DataFrame(ediya)
df.to_csv('ediya.csv', index=False, encoding='utf-8')


/var/folders/13/lngvcc9153b7xk5c_vj9hm2r0000gn/T/ipykernel_24086/958386171.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../chromedriver.exe') # 이건 우리 빅데이터 강의 자료 6장 20p 보면 다운받는 url있음


[{'name': '강남YMCA점', 'address': '서울 강남구 논현동', 'gu': '강남구'},
 {'name': '강남구청역아이티웨딩점',
  'address': '서울 강남구 학동로 338 (논현동, 강남파라곤)',
  'gu': '강남구'},
 {'name': '강남논현학동점', 'address': '서울 강남구 논현로131길 28 (논현동)', 'gu': '강남구'},
 {'name': '강남대치점', 'address': '서울 강남구 역삼로 415 (대치동, 성진빌딩)', 'gu': '강남구'},
 {'name': '강남도산점', 'address': '서울 강남구 도산대로37길 20 (신사동)', 'gu': '강남구'},
 {'name': '강남율현점', 'address': '서울 강남구 밤고개로21길 8 (율현동, 세곡프라자)', 'gu': '강남구'},
 {'name': '강남자곡점', 'address': '서울 강남구 자곡로 180 (자곡동, 강남유탑유블레스)', 'gu': '강남구'},
 {'name': '개포동역점', 'address': '서울 강남구 개포로82길 11 (개포동, 삼우빌딩)', 'gu': '강남구'},
 {'name': '건설회관점', 'address': '서울 강남구 언주로 711 (논현동)', 'gu': '강남구'},
 {'name': '대청역점', 'address': '서울 강남구 개포로109길 34 (개포동, 대청프라자)', 'gu': '강남구'},
 {'name': '도곡2동점', 'address': '서울 강남구 논현로 142 (도곡동, 동현빌딩)', 'gu': '강남구'},
 {'name': '도산사거리점', 'address': '서울 강남구 논현동 ', 'gu': '강남구'},
 {'name': '삼성대치점', 'address': '서울 강남구 삼성로64길 32 (대치동, 주성빌딩)', 'gu': '강남구'},
 {'name': '삼성동점', 'address': '서울 강남구 테헤란로88길 10 (대치